In [4]:
%run "./Includes/Classroom-Setup"

### A Step Toward Full Automation

Notebooks are one--and not the only--way of interacting with the Spark and Databricks environment.  Notebooks can be executed independently and as recurring jobs.  They can also be exported and versioned using git.  Python files and Scala/Java jars can be executed against a Databricks cluster as well, allowing full integration with a developer's normal workflow.  Since notebooks can be executed like code files and compiled binaries, they offer a way of building production pipelines.

Functional programming design principles aid in thinking about pipelines.  In functional programming, your code always has known inputs and outputs without any side effects.  In the case of automating notebooks, coding notebooks in this way helps reduce any unintended side effects where each stage in a pipeline can operate independently from the rest.

More complex workflows using notebooks require managing dependencies between tasks and passing parameters into notebooks.  Dependency management can done by chaining notebooks together, for instance to run reporting logic after having completed a database write. Sometimes, when these pipelines become especially complex, chaining notebooks together isn't sufficient. In those cases, scheduling with Apache Airflow has become the preferred solution. Notebook widgets can be used to pass parameters to a notebook when the parameters are determined at runtime.

### Widgets

Widgets allow for the customization of notebooks without editing the code itself.  They also allow for passing parameters into notebooks.  There are 4 types of widgets:

| Type          | Description                                                                                        |
|:--------------|:---------------------------------------------------------------------------------------------------|
| `text`        | Input a value in a text box.                                                                       |
| `dropdown`    | Select a value from a list of provided values.                                                     |
| `combobox`    | Combination of text and dropdown. Select a value from a provided list or input one in the text box.|
| `multiselect` | Select one or more values from a list of provided values.                                          |

Widgets are Databricks utility functions that can be accessed using the `dbutils.widgets` package and take a name, default value, and values (if not a `text` widget).

In [8]:
dbutils.widgets.dropdown("MyWidget", "1", [str(x) for x in range(1, 5)])

Notice the widget created at the top of the screen.  Choose a number from the dropdown menu.  Now, bring that value into your code using the `get` method.

In [10]:
dbutils.widgets.get("MyWidget")

Out[16]: '1'

Clear the widgets using either `remove()` or `removeAll()`

In [12]:
dbutils.widgets.removeAll()

While great for adding parameters to notebooks and dashboards, widgets also allow us to pass parameters into notebooks when we run them like a Python or JAR file.

### Running Notebooks

There are two options for running notebooks.  The first is using `dbutils.notebook.run("<path>", "<timeout>")`.  

Now run the notebook with the following command.

In [ ]:
return_value = dbutils.notebook.run("./Runnable/Runnable-1", 30)

print("Notebook successfully ran with return value: {}".format(return_value))

Notice how the `Runnable-1` notebook ends with the command `dbutils.notebook.exit("returnValue")`.  This is a `string` that's passed back into the running notebook's environment.

Run the following cell and note how the variable doesn't exist.

This variable is not passed into our current environment.  The difference between `dbutils.notebook.run()` and `%run` is that the parent notebook will inherit variables from the ran notebook with `%run`.

In [20]:
%run ./Runnable/Runnable-1

Now this variable is available for use in this notebook

In [22]:
print(my_variable)

12

### Parameter Passing and Debugging

Notebook widgets allow us to pass parameters into notebooks.  This can be done in the form of a dictionary that maps the widget name to a value as a `string`.


Pass your parameters into `dbutils.notebook.run` and save the resulting return value

In [ ]:
basePath =  "{}/etl3p/".format(getUserhome())
dest_path = "{}/academy/raw_logs.parquet".format(basePath)

result = dbutils.notebook.run("./Runnable/Runnable-2", 60, {"date": "11-27-2013", "dest_path": dest_path})

Click on `Notebook job #XXX` above to view the output of the notebook.  **This is helpful for debugging any problems.**

Parse the JSON string results

In [ ]:
import json
print(json.loads(result))

Now look at what this accomplished: cell phone logs were parsed corresponding to the date of the parameter passed into the notebook.  The results were saved to the given destination path.

In [ ]:
display(spark.read.parquet(dest_path))

### Dependency Management, Timeouts, Retries, and Passing Data

Running notebooks can allow for more advanced workflows in the following ways:<br><br>

* Managing **dependencies** can be ensured by running a notebook that triggers other notebooks in the desired order
* Setting **timeouts** ensures that jobs have a set limit on when they must either complete or fail
* **Retry logic** ensures that fleeting failures do not prevent the proper execution of a notebook
* **Data can passed** between notebooks by saving the data to a blob store or table and passing the path as an exit parameter


## Exercise 1: Building a Generalized Notebook

Build a notebook that allows for customization using input parameters

### Step 1: Filter an Hour of Log Data

1. Takes two parameters: `hour` and `output_path`
1. Reads the following log file: `/mnt/training/EDGAR-Log-20170329/enhanced/EDGAR-Log-20170329-sample.parquet`
1. Filters the data for the hour provided
1. Writes the result to the `output_path`
1. Exits with the `output_path` as the exit parameter

In [ ]:
path = "{}/hour_03.parquet".format(basePath)

dbutils.notebook.run("./Runnable/Runnable-3", 60, {"hour": "03", "output_path": path})

In [35]:
# TEST - Run this cell to test your solution
import random

r = str(random.randint(0, 10**10))
_path = "{}/hour_08_{}.parquet".format(basePath, r)

_returnValue = dbutils.notebook.run("./Runnable/Runnable-3", 60, {"hour": "08", "output_path": _path})
_df = spark.read.parquet(_returnValue)

dbTest("ET3-P-03-01-01", True, _path == _returnValue)
dbTest("ET3-P-03-01-02", 54206, _df.count())

dbutils.fs.rm(_path, True)

print("Tests passed!")

## Review
**Question:** How can I start to transition from notebooks to production environments?  
**Answer:** Runnable notebooks are the first step towards transitioning into production environments since they allow us to generalize and parameterize our code.  There are other options, including running Python files and Scala/Java jars against a cluster as well.

**Question:** How does passing parameters into and out of notebooks work?  
**Answer:** Widgets allow for the customization of notebooks and passing parameters into them.  This takes place in the form of a dictionary (Python) or map (Scala) of key/value pairs that match the names of widgets.  Only strings can be passed out of a notebook as an exit parameter.

**Question:** Since I can only pass strings of a limited length out of a notebook, how can I pass data out of a notebook?  
**Answer:** The preferred way is to save your data to a blob store or Spark table.  On the notebook's exit, pass the location of that data as a string.  It can then be easily imported and manipulated in another notebook.

In [38]:
%run "./Includes/Classroom-Cleanup"